In [5]:
import pandas as pd

df = pd.read_csv('metro.tsv', sep='\t', encoding='utf-8')

df.head()

,Название,Широта,Долгота
0,Аэропорт,55.79981,37.53412
1,Академическая,55.68808,37.57501
2,Алексеевская,55.80737,37.63844
3,Александровский сад,55.75219,37.60836
4,Алтуфьево,55.89504,37.58605


In [6]:
def get_icons_per_district():
	icon_format = "https://domokucha.info/img/V_{0}_of_Moscow_coa.png"

	import requests
	from bs4 import BeautifulSoup
	url = 'https://domokucha.info/moscow_metro_district.html'

	response = requests.get(url)
	soup = BeautifulSoup(response.content, 'html.parser')

	for row in soup.select_one('.wrapper .container').select_one('section').select('.features-item'):
		district_name = next(row.select_one('.features-title').children, None)
		icon_url = icon_format.format(row.select_one('.features-icon')['class'][1].removeprefix('features-icon-V'))
		
		yield district_name.__str__().strip(), icon_url

In [7]:
def find_district_from_geojson(geojson, lat, lng): 
	import json
	from shapely.geometry import shape, Point

	with open(geojson, encoding='utf-8') as f:
		data = json.load(f)

	point = Point(lng, lat)

	for feature in data['features']:
		polygon = shape(feature['geometry'])
		if polygon.contains(point):
			return feature['properties']['NAME']

	return None

In [8]:
dict_icons = dict(get_icons_per_district())
df['district'] = df.apply(lambda row: find_district_from_geojson('ao.geojson', row['Широта'], row['Долгота']), axis=1)
df['icon'] = df['district'].map(dict_icons)

In [9]:
dict_icons

{'Восточный': 'https://domokucha.info/img/V_VAO_district_of_Moscow_coa.png',
 'Западный': 'https://domokucha.info/img/V_ZAO_district_of_Moscow_coa.png',
 'Северный': 'https://domokucha.info/img/V_SAO_district_of_Moscow_coa.png',
 'Северо-Западный': 'https://domokucha.info/img/V_SZAO_district_of_Moscow_coa.png',
 'Северо-Восточный': 'https://domokucha.info/img/V_SVAO_district_of_Moscow_coa.png',
 'Центральный': 'https://domokucha.info/img/V_CAO_district_of_Moscow_coa.png',
 'Юго-Восточный': 'https://domokucha.info/img/V_UVAO_district_of_Moscow_coa.png',
 'Юго-Западный': 'https://domokucha.info/img/V_UZAO_district_of_Moscow_coa.png',
 'Южный': 'https://domokucha.info/img/V_UAO_district_of_Moscow_coa.png',
 'Зеленоградский': 'https://domokucha.info/img/V_Zelenograd_district_of_Moscow_coa.png',
 'Новомосковский': 'https://domokucha.info/img/V_new-moscow_district_of_Moscow_coa.png',
 'Троицкий': 'https://domokucha.info/img/V_troicky_district_of_Moscow_coa.png'}

In [10]:
df

,Название,Широта,Долгота,district,icon
0,Аэропорт,55.799810,37.534120,Северный,https://domokucha.info/img/V_SAO_district_of_M...
1,Академическая,55.688080,37.575010,Юго-Западный,https://domokucha.info/img/V_UZAO_district_of_...
2,Алексеевская,55.807370,37.638440,Северо-Восточный,https://domokucha.info/img/V_SVAO_district_of_...
3,Александровский сад,55.752190,37.608360,Центральный,https://domokucha.info/img/V_CAO_district_of_M...
4,Алтуфьево,55.895040,37.586050,Северо-Восточный,https://domokucha.info/img/V_SVAO_district_of_...
...,...,...,...,...,...
160,Воробьёвы горы,55.710454,37.558601,Центральный,https://domokucha.info/img/V_CAO_district_of_M...
161,Выхино,55.715000,37.818020,Юго-Восточный,https://domokucha.info/img/V_UVAO_district_of_...
162,Ясенево,55.605350,37.534940,Юго-Западный,https://domokucha.info/img/V_UZAO_district_of_...
163,Юго-западная,55.664640,37.484210,Западный,https://domokucha.info/img/V_ZAO_district_of_M...


In [11]:
import folium

m = folium.Map(location=[df['Широта'].mean(), df['Долгота'].mean()], zoom_start=10)


for idx, row in df.iterrows():
	folium.Marker(
		icon=folium.CustomIcon(
			icon_image=row['icon'],
			 icon_size=(45, 45),
		),
		location=[row['Широта'], row['Долгота']],
		popup='<b>Станция: {}</b><br>Район: {}<br> {};{}'.format(row['Название'], row['district'], row['Широта'], row['Долгота']),
	).add_to(m)

m